# Assignment 2
# Syed Saad Ullah Shah
# 400202

## CV - MSDS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [ ]:
!gdown https://drive.google.com/uc?id=103XRftSJE_qsVl3-GxmbEjOFcOJuUEvI

Downloading...
From (uriginal): https://drive.google.com/uc?id=103XRftSJE_qsVl3-GxmbEjOFcOJuUEvI
From (redirected): https://drive.google.com/uc?id=103XRftSJE_qsVl3-GxmbEjOFcOJuUEvI&confirm=t&uuid=2c560cf3-a316-4bd1-b27f-b09e1cc80881
To: /content/train_and_val_set.tar
100% 3.78G/3.78G [00:23<00:00, 160MB/s]


In [ ]:
!tar -xvf train_and_val_set.tar

Streaming output truncated to the last 5000 lines.
train_set/annotations/392066_lnd.npy
train_set/annotations/83536_lnd.npy
train_set/annotations/26361_lnd.npy
train_set/annotations/313969_aro.npy
train_set/annotations/286825_exp.npy
train_set/annotations/392215_aro.npy
train_set/annotations/330087_lnd.npy
train_set/annotations/110001_val.npy
train_set/annotations/371482_aro.npy
train_set/annotations/27940_aro.npy
train_set/annotations/35729_exp.npy
train_set/annotations/156731_aro.npy
train_set/annotations/145081_aro.npy
train_set/annotations/93043_exp.npy
train_set/annotations/279607_lnd.npy
train_set/annotations/48472_lnd.npy
train_set/annotations/206512_exp.npy
train_set/annotations/84264_aro.npy
train_set/annotations/187769_lnd.npy
train_set/annotations/300707_exp.npy
train_set/annotations/123766_exp.npy
train_set/annotations/173216_aro.npy
train_set/annotations/262096_lnd.npy
train_set/annotations/194946_aro.npy
train_set/annotations/411301_aro.npy
train_set/annotations/43103_aro

In [ ]:
data_path = "/content/train_set/"
data_annotations_path = "/content/train_set/annotations/"
data_images_path = "/content/train_set/images/"
# data_annotations_val_path = "/content/val_set/annotations/"
# data_images_val_path = "/content/val_set/images/"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd
import torchvision
from tqdm import tqdm
import os

# Data Preparation



### Loading dataset and transforming it into *DataFrames*



In [ ]:
def dataset_tp_csv(data_annotations_path, data_images_path):
  data = []
  for i in range(200000):
      try:
          arrousal = np.load(data_annotations_path + '{index}_aro.npy'.format(index=i))
      except FileNotFoundError:
          arrousal = None
          
      try:
          expression = np.load(data_annotations_path + '{index}_exp.npy'.format(index=i))
      except FileNotFoundError:
          expression = None
          
      try:
          valence = np.load(data_annotations_path + '{index}_val.npy'.format(index=i))
      except FileNotFoundError:
          valence = None
          
      try:
          landmarks = np.load(data_annotations_path + '{index}_lnd.npy'.format(index=i))
      except FileNotFoundError:
          landmarks = None
          
      image_path = '{path}/{index}.jpg'.format(path=data_images_path, index=i)
      if os.path.isfile(image_path):
          data.append((i, expression, valence, arrousal, landmarks, image_path))
      else:
          print('Image file not found for index', i)
  return pd.DataFrame.from_records(data, columns=['image_index', 'expression', 'valence', 'arousal', 'landmarks', 'image_path'])

In [ ]:
df_train = dataset_tp_csv(data_annotations_path, data_images_path)

Streaming output truncated to the last 5000 lines.
Image file not found for index 183808
Image file not found for index 183810
Image file not found for index 183811
Image file not found for index 183814
Image file not found for index 183815
Image file not found for index 183823
Image file not found for index 183827
Image file not found for index 183828
Image file not found for index 183829
Image file not found for index 183830
Image file not found for index 183832
Image file not found for index 183834
Image file not found for index 183840
Image file not found for index 183842
Image file not found for index 183843
Image file not found for index 183867
Image file not found for index 183868
Image file not found for index 183872
Image file not found for index 183876
Image file not found for index 183884
Image file not found for index 183889
Image file not found for index 183896
Image file not found for index 183898
Image file not found for index 183901
Image file not found for index 183902

In [ ]:
# df_val = dataset_tp_csv(data_annotations_val_path, data_images_val_path)

### Transforming Dataframes into *Dataloaders*

In [ ]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define dataset class
class FacialExpressionDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
    
    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'image_path']
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        # landmarks = self.df.loc[idx, 'landmarks']
        # landmarks = torch.tensor(self.df.loc[idx, 'landmarks'], dtype=torch.float32)
        # expression = self.df.loc[idx, 'expression']
        expression_arr = int(self.df.loc[idx, 'expression'].item())
        expression = torch.tensor(expression_arr, dtype=torch.int32)
        # valence = self.df.loc[idx, 'valence']
        valence_arr = float(self.df.loc[idx, 'valence'].item())
        valence = torch.tensor(valence_arr, dtype=torch.float32)
        # arousal = self.df.loc[idx, 'arousal']
        arousal_arr = float(self.df.loc[idx, 'arousal'].item())
        arousal = torch.tensor(arousal_arr, dtype=torch.float32)
        # return img, landmarks, expression, valence, arousal
        return img, expression, valence, arousal
    
    def __len__(self):
        return len(self.df)

In [ ]:
batch_size = 32

data_path = "/content/drive/MyDrive/Nust/Semester 2/Computer Vision/Assignments/Assignment 2/sample_train_set/"
data_val_path = "/content/drive/MyDrive/Nust/Semester 2/Computer Vision/Assignments/Assignment 2/sample_val_set/"

# Define the AffectNet dataset and dataloaders
train_dataset = FacialExpressionDataset(df_train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# val_dataset = FacialExpressionDataset(df_val, transform=transform)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# FineTuning Models for Facial Expression recognition, and computing Valence and Arousal Task

### 1):EffecientNET

In [ ]:
# Define the EfficientNet model
class EfficientNetModel(nn.Module):
    def __init__(self, num_classes=8):
        super(EfficientNetModel, self).__init__()
        self.num_classes = num_classes
        # self.efficientnet = EfficientNet.from_pretrained('efficientnet-b3')
        # x = self.features(x)
        # x = self.avgpool(x)
        # x = torch.flatten(x, 1)
        # x = self.classifier(x)

        self.layer1 = torchvision.models.efficientnet_b0(pretrained=True).features
        self.layer2 = torchvision.models.efficientnet_b0(pretrained=True).avgpool

        # self.efficientnet._fc = nn.Linear(1536, num_classes)

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(1280, 8),
        )

        self.arousal = nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(1280, 1),
        )

        self.valence = nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(1280, 1),
        )

        # self.valence_fc = nn.Sequential(
        #     nn.Linear(1536, 256),
        #     nn.ReLU(inplace=True),
        #     nn.Linear(256, 1),
        # )

        # self.arousal_fc = nn.Sequential(
        #     nn.Linear(1536, 256),
        #     nn.ReLU(inplace=True),
        #     nn.Linear(256, 1),
        # )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        # logits = self.efficientnet._fc(x)
        feature = torch.flatten(x, 1)
        logits = self.classifier(feature)

        # x = self.efficientnet(x)
        # logits = x
        # valence = self.valence_fc(feature)
        # arousal = self.arousal_fc(feature)

        valence = self.valence(feature)
        arousal = self.arousal(feature)

        return logits, valence, arousal

In [ ]:
# Define the training function
def train(model, device, train_loader, criterion, criterion1, optimizer):
    model.train()
    train_loss = 0
    for data, target, valence, arousal in tqdm(train_loader, desc="Training"):
        data, target, valence, arousal = data.to(device), target.to(device), valence.to(device), arousal.to(device)
        target = target.type(torch.LongTensor).to(device)
        # valence = valence.type(torch.LongTensor).to(device)
        # arousal = arousal.type(torch.LongTensor).to(device)
        optimizer.zero_grad()
        logits, pred_valence, pred_arousal = model(data)
        loss = criterion(logits, target) + criterion1(pred_valence.squeeze(), valence) + criterion1(pred_arousal.squeeze(), arousal)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader.dataset)

    return train_loss


In [ ]:
# Define the training parameters
batch_size = 32
num_epochs = 15
learning_rate = 0.0001

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model, optimizer, and loss function
model = EfficientNetModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)
criterion_mse = nn.MSELoss().to(device)

# Train the model
for epoch in range(num_epochs):
    train_loss = train(model, device, train_loader, criterion, criterion_mse, optimizer)
    print(f"Epoch {epoch + 1}/{num_epochs}: Train Loss: {train_loss:.4f}")

# Save the model
torch.save(model.state_dict(), "EfficientNet_model.pth")


Training: 100%|██████████| 4326/4326 [09:05<00:00,  7.93it/s]


Epoch 1/15: Train Loss: 0.0316


Training: 100%|██████████| 4326/4326 [08:18<00:00,  8.68it/s]


Epoch 2/15: Train Loss: 0.0256


Training: 100%|██████████| 4326/4326 [07:54<00:00,  9.11it/s]


...

Epoch 13/15: Train Loss: 0.0074


Training: 100%|██████████| 4326/4326 [08:15<00:00,  8.73it/s]


Epoch 14/15: Train Loss: 0.0071


Training: 100%|██████████| 4326/4326 [08:13<00:00,  8.77it/s]


Epoch 15/15: Train Loss: 0.0066


In [ ]:
from torchsummary import summary

model=EfficientNetModel().to(device)
x=torch.rand(size=[128,3, 224, 224])
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

### 2): Mobile Net V3

In [ ]:
# Define the EfficientNet model
class MobileNetModel(nn.Module):
    def __init__(self, num_classes=8):
        super(MobileNetModel, self).__init__()
        self.num_classes = num_classes

        self.layer1 = torchvision.models.mobilenet_v3_small(pretrained=True).features
        self.layer2 = torchvision.models.mobilenet_v3_small(pretrained=True).avgpool

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(576, 8),
        )

        self.arousal = nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(576, 1),
        )

        self.valence = nn.Sequential(
            nn.Dropout(p=0.2, inplace=False),
            nn.Linear(576, 1),
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        feature = torch.flatten(x, 1)
        logits = self.classifier(feature)
        valence = self.valence(feature)
        arousal = self.arousal(feature)

        return logits, valence, arousal

In [ ]:
# Define the training function
def train(model, device, train_loader, criterion, criterion1, optimizer):
    model.train()
    train_loss = 0
    for data, target, valence, arousal in tqdm(train_loader, desc="Training"):
        data, target, valence, arousal = data.to(device), target.to(device), valence.to(device), arousal.to(device)
        target = target.type(torch.LongTensor).to(device)
        optimizer.zero_grad()
        logits, pred_valence, pred_arousal = model(data)
        loss = criterion(logits, target) + criterion1(pred_valence.squeeze(), valence) + criterion1(pred_arousal.squeeze(), arousal)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader.dataset)

    return train_loss


In [ ]:
# Define the training parameters
batch_size = 32
num_epochs = 15
learning_rate = 0.0001

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model, optimizer, and loss function
model = MobileNetModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)
criterion_mse = nn.MSELoss().to(device)

# Train the model
for epoch in range(num_epochs):
    train_loss = train(model, device, train_loader, criterion, criterion_mse, optimizer)
    print(f"Epoch {epoch + 1}/{num_epochs}: Train Loss: {train_loss:.4f}")

# Save the model
torch.save(model.state_dict(), "MobileNet_model.pth")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 126MB/s]
Training: 100%|██████████| 4326/4326 [05:45<00:00, 12.52it/s]


Epoch 1/15: Train Loss: 0.0341


Training: 100%|██████████| 4326/4326 [05:50<00:00, 12.33it/s]


Epoch 2/15: Train Loss: 0.0279


Training: 100%|██████████| 4326/4326 [05:52<00:00, 12.28it/s]


...

Epoch 13/15: Train Loss: 0.0149


Training: 100%|██████████| 4326/4326 [05:43<00:00, 12.58it/s]


Epoch 14/15: Train Loss: 0.0139


Training: 100%|██████████| 4326/4326 [05:44<00:00, 12.57it/s]

Epoch 15/15: Train Loss: 0.0129


In [ ]:
from torchsummary import summary

model=MobileNetModel().to(device)
x=torch.rand(size=[128,3, 224, 224])
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
         Hardswish-3         [-1, 16, 112, 112]               0
            Conv2d-4           [-1, 16, 56, 56]             144
       BatchNorm2d-5           [-1, 16, 56, 56]              32
              ReLU-6           [-1, 16, 56, 56]               0
 AdaptiveAvgPool2d-7             [-1, 16, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             136
              ReLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 16, 1, 1]             144
      Hardsigmoid-11             [-1, 16, 1, 1]               0
SqueezeExcitation-12           [-1, 16, 56, 56]               0
           Conv2d-13           [-1, 16, 56, 56]             256
      BatchNorm2d-14           [-1, 16,